In [1]:
from collections import OrderedDict

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus

from helpers import create_models

In [2]:
DIR = 'TEST_DATA/DENSE'
Q_SEED = 10598 # as used in most issues
aqua_globals.random_seed = Q_SEED
shots = 4096

In [3]:
qps = create_models(DIR)
optimizer = COBYLA() 
cplex = CplexOptimizer()

backend = BasicAer.get_backend('qasm_simulator')
max_qubits = backend.configuration().to_dict()['n_qubits']

quantum_instance = QuantumInstance(backend,
                                   seed_simulator=Q_SEED,
                                   seed_transpiler=Q_SEED,
                                   shots=shots) 
qaoa_mes = QAOA(quantum_instance=quantum_instance, optimizer=optimizer)

In [4]:
results = OrderedDict()
for qp_name, qp in qps.items() :
    print(qp_name)
    print("number of qubits: ", qp.qubo.get_num_vars())
    
    if qp.qubo.get_num_vars() > max_qubits:
        break
        
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    res = qaoa.solve(qp)
    results[qp_name] = res
    if res.status == OptimizationResultStatus.SUCCESS:
        print("success")
        cres  = cplex.solve(qp)
        if res.fval != cres.fval:
            print("\n optimal value QAOA   "+str(res.fval) + "  ,  cplex:"+ str(cres.fval))

# see Class SubstituteVariables in https://qiskit.org/documentation/_modules/qiskit/optimization/problems/quadratic_program.html#QuadraticProgram
# for logger warning constraint A0_leb0 is infeasible due to substitution
# Afterwards a check for feasability of substituted solution is done (line 227 in meo result class) to set OptimizationResultStatus

test_4
number of qubits:  7
success
test_3
number of qubits:  8
success
test_5
number of qubits:  13
success
test_6
number of qubits:  17
success
test_7
number of qubits:  18
success

 optimal value QAOA   -3.0  ,  cplex:-5.0
test_8
number of qubits:  22


constraint A0_leb3 is infeasible due to substitution


success

 optimal value QAOA   -2.0  ,  cplex:-4.0
test_9
number of qubits:  25


In [5]:
res_7 = results["test_7"]
print(res_7)

optimal function value: -3.0
optimal value: [1. 0. 1. 1. 0. 1. 0.]
status: SUCCESS


In [6]:
qps["test_7"].is_feasible(res_7.x)

True

In [7]:
qps["test_8"].is_feasible(results["test_8"].x)

True

In [8]:
qps["test_8"].to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: test_8
// single vars section
dvar bool x0_0;
dvar bool x0_2;
dvar bool x0_3;
dvar bool x0_4;
dvar bool x0_6;
dvar bool x0_1;
dvar bool x0_7;
dvar bool x0_5;

minimize
 - 2 x0_0 + 2 x0_2 + x0_3 - x0_4 - x0_6;
 
subject to {
 A0_leb0:
  2 x0_0 - 3 x0_2 - x0_4 + x0_6 - x0_1 - 3 x0_7 <= 1;
 A0_leb1:
  - 2 x0_0 - x0_2 - 2 x0_3 + 2 x0_4 - 2 x0_6 - 2 x0_1 + 3 x0_5 <= 3;
 A0_leb2:
  - 2 x0_2 - 3 x0_3 - 2 x0_4 - 3 x0_6 + 3 x0_7 - 2 x0_5 <= -6;
 A0_leb3:
  - 3 x0_0 + 3 x0_3 + x0_4 - 2 x0_6 + 2 x0_1 + 2 x0_7 + x0_5 <= 2;

}
